<a href="https://colab.research.google.com/github/Upparapallivenkatesh/FMML-ASSESSMENTS-/blob/main/Copy_of_Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Yes, averaging the validation accuracy across multiple splits in techniques like k-fold cross-validation can indeed give more consistent and reliable results compared to using a single validation set. Here's why:

Reduced Variance: When you split your dataset into multiple folds (typically k folds), you create several subsets of data. By training and evaluating your model k times, each time on a different fold, you reduce the impact of randomness in the data splitting process. This helps in reducing the variance of your evaluation metric (e.g., accuracy) and provides a more stable estimate of your model's performance.

Better Generalization: Averaging the results over multiple folds allows your model to be tested on different subsets of the data, making it more robust and better at generalizing to unseen data. This helps you get a more realistic estimate of how your model will perform on new, unseen examples.

Lower Risk of Overfitting: Using a single fixed validation set can lead to overfitting if you make repeated adjustments to your model based on its performance on that specific validation set. By using multiple validation sets, you reduce the risk of overfitting to a single set of validation data.

More Information: Cross-validation provides more information about your model's performance. You not only get a single accuracy score but a distribution of scores. This additional information can be valuable in understanding how stable and consistent your model's performance is.

Better Hyperparameter Tuning: Cross-validation is often used in hyperparameter tuning (e.g., grid search or random search) to find the best set of hyperparameters for your model. Averaging results over multiple splits helps in selecting hyperparameters that perform well across different subsets of the data.

In summary, averaging validation accuracy across multiple splits in cross-validation is a good practice to obtain a more robust and consistent estimate of your model's performance. It is widely used in machine learning to assess model performance and make data-driven decisions about model selection and hyperparameter tuning.






Cross-validation provides a more accurate estimate of how your model is likely to perform on unseen data compared to a single fixed validation set, but it's important to understand the distinction between "test accuracy" and "validation accuracy."

Validation Accuracy: When you perform cross-validation, you partition your dataset into multiple subsets and train and validate your model on these subsets. The validation accuracy obtained from cross-validation gives you an estimate of how well your model is likely to perform on new, unseen data that is similar to your training data.

Test Accuracy: Test accuracy, on the other hand, is typically obtained by using a completely separate dataset that was not used during model training or hyperparameter tuning. This dataset is truly "unseen" and is used to provide a final assessment of your model's performance.

While cross-validation provides a more accurate estimate of your model's performance on similar data to your training data, it doesn't replace the need for a separate test set. The test set provides an unbiased evaluation of your model's generalization to completely new and unseen data, which is the ultimate goal in machine learning.

Cross-validation is valuable for model selection, hyperparameter tuning, and assessing how well your model generalizes within the dataset you have. It helps you avoid overfitting to a single validation set. However, test accuracy remains the gold standard for estimating how your model will perform in real-world, production scenarios.

In practice, you would use cross-validation to make informed decisions about your model's architecture and hyperparameters and then assess your model's final performance using a separate test set. This two-step process ensures that you have a reliable estimate of both your model's performance on similar data and its generalization to unseen data.






The number of iterations, or folds, in a cross-validation process can have an impact on the quality of the estimate you obtain. However, there is a trade-off between computational cost and the reliability of the estimate:

Fewer Iterations (Folds): If you use a small number of iterations (e.g., 2 or 3), you may get a quicker estimate of your model's performance, but it can be more sensitive to the particular random split of the data. The estimate might have higher variance, and you might not capture the full diversity of your dataset in the validation process.

More Iterations (Folds): Increasing the number of iterations, such as using k-fold cross-validation with a higher value of k, can lead to a more reliable estimate of your model's performance. This is because you're averaging the results over multiple subsets of data, which helps reduce the impact of random variability in the data splitting process. A larger value of k is generally considered to provide a more stable estimate.

However, there are some considerations to keep in mind:

Computational Cost: As you increase the number of iterations, the computational cost of cross-validation also increases. Training and evaluating the model k times can be time-consuming, especially with large datasets or complex models.

Diminishing Returns: There are diminishing returns with increasing k. Beyond a certain point, increasing k may not significantly improve the estimate of your model's performance, but it will increase computational overhead.

Data Size: The number of iterations should also be chosen with consideration of the size of your dataset. If you have a very small dataset, using a high k value might result in each fold having very little data, which can lead to unreliable estimates. In such cases, stratified sampling or leave-one-out cross-validation might be more appropriate.

In practice, the choice of the number of iterations in cross-validation is often determined by a balance between computational resources and the desire for a reliable estimate. Common choices include 5-fold or 10-fold cross-validation, which strike a reasonable balance between accuracy and computational efficiency for many datasets. However, it's important to adapt the number of iterations to the specific characteristics of your data and the available computational resources.






Increasing the number of iterations in cross-validation can help mitigate some of the challenges posed by having a very small training dataset or validation dataset, but it does have limitations. Here's how increasing iterations can affect the handling of small datasets:

Small Training Dataset:

Benefit of Increasing Iterations: When you have a small training dataset, increasing the number of iterations (e.g., using a higher k value in k-fold cross-validation) can be beneficial. It allows your model to train on different subsets of the small training data multiple times, which can help in capturing a more comprehensive understanding of the data's variability.

More Reliable Estimates: With more iterations, you get a more reliable estimate of your model's performance because you're averaging the results over multiple training-validation splits, reducing the impact of randomness in the data splitting process.

Limitations: However, there are limitations to how much you can compensate for a very small training dataset. If your training dataset is extremely small, even with a high number of iterations, each fold might still have very limited data to learn from. This can lead to unstable estimates and potential overfitting to the small training dataset.

Small Validation Dataset:

Benefit of Increasing Iterations: Increasing the number of iterations can also help when you have a small validation dataset. In k-fold cross-validation, each fold serves as both a training and validation set. When the validation dataset is small, having more iterations (higher k) means that you'll be validating your model's performance on different subsets of the small validation data, which can provide a more robust estimate.

Improved Robustness: More iterations can make your evaluation more robust because it reduces the influence of a small, potentially unrepresentative validation set on your model's performance estimate.

Limitations: Just like with a small training dataset, there are limits to how much you can compensate for a very small validation dataset. If your validation dataset is extremely small, increasing iterations can help but might not completely eliminate the limitations of having insufficient validation data.

In summary, increasing the number of iterations in cross-validation is a useful strategy to improve the robustness and reliability of your model evaluation, especially when dealing with small datasets. However, there are inherent limitations to what can be achieved with very small datasets, and it's essential to strike a balance between the number of iterations and the size of the dataset based on the specific circumstances and computational resources available.





